# Repro old results: No site rates in method nor in simulation

In [1]:
%%time
from src.pipeline import Pipeline
from src.end_to_end_simulation import EndToEndSimulator
from collections import defaultdict

experiment_rootdir = "cache_LG"

def run_experiment_and_return_learned_rate_matrices():
    learned_rate_matrices = defaultdict(dict)
    num_transitions = defaultdict(dict)
    for max_families in max_families_list:
        pipeline = Pipeline(
            outdir=f"{experiment_rootdir}/pipeline",
            max_seqs=100000,
            max_sites=100000,
            armstrong_cutoff=None,
            rate_matrix='input_data/synthetic_rate_matrices/WAG_FastTree.txt',
            n_process=32,
            expected_number_of_MSAs=3912,
            max_families=max_families,
            a3m_dir='input_data/LG_Train_a3m',
            pdb_dir=None,
            use_cached=True,
            num_epochs=200,
            device='cpu',
            center=0.06,
            step_size=0.1,
            n_steps=50,
            keep_outliers=False,
            max_height=1000.0,
            max_path_height=1000,
            precomputed_contact_dir=None,
            precomputed_tree_dir=None,
            precomputed_maximum_parsimony_dir=None,
            edge_or_cherry="cherry",
            init_jtt_ipw=True,
            method=["MLE"],
            xrate_grammar=None,
            fast_tree_cats=4,
            use_site_specific_rates=False,
        )
        pipeline.run()
        learned_rate_matrices['real_data'][max_families] = pipeline.get_learned_Q1()
        num_transitions['real_data'][max_families] = pipeline.get_number_of_single_site_transitions()

        end_to_end_simulator = EndToEndSimulator(
            outdir=f"{experiment_rootdir}/end_to_end_simulator",
            pipeline=pipeline,
            simulation_pct_interacting_positions=0.0,
            use_site_specific_rates_in_simulation=False,
            Q1_ground_truth="input_data/synthetic_rate_matrices/WAG_matrix.txt",
            Q2_ground_truth="input_data/synthetic_rate_matrices/Q2_uniform_constrained.txt",
            fast_tree_rate_matrix='input_data/synthetic_rate_matrices/WAG_FastTree.txt',
            fast_tree_cats=4,
            use_site_specific_rates=False,
            simulate_end_to_end=True,
            simulate_from_trees_wo_ancestral_states=True,
            simulate_from_trees_w_ancestral_states=True,
            use_cached=True,
        )
        end_to_end_simulator.run()
        learned_rate_matrices['w_ancestral_states'][max_families] = end_to_end_simulator.pipeline_on_simulated_data_from_trees_w_ancestral_states.get_learned_Q1()
        learned_rate_matrices['wo_ancestral_states'][max_families] = end_to_end_simulator.pipeline_on_simulated_data_from_trees_wo_ancestral_states.get_learned_Q1()
        learned_rate_matrices['end_to_end'][max_families] = end_to_end_simulator.pipeline_on_simulated_data_end_to_end.get_learned_Q1()
    return learned_rate_matrices, num_transitions
max_families_list = [3912]
learned_rate_matrices_orig, num_transitions_orig = run_experiment_and_return_learned_rate_matrices()

100%|██████████| 3912/3912 [00:00<00:00, 12465.44it/s]

CPU times: user 6.54 s, sys: 10.6 s, total: 17.1 s
Wall time: 11.8 s


# Break old method: no site rates in method, yes in simulations

# Only caveat is that because we use site-specific rates in simulation, only 1024 sites will be used per family, making the simulated data potentially smaller for families larger that 1024. But I expect deteriorated performance to be due to not accounting for site rates

In [2]:
%%time
from src.pipeline import Pipeline
from src.end_to_end_simulation import EndToEndSimulator
from collections import defaultdict

experiment_rootdir = "cache_LG"

def run_experiment_and_return_learned_rate_matrices():
    learned_rate_matrices = defaultdict(dict)
    num_transitions = defaultdict(dict)
    for max_families in max_families_list:
        pipeline = Pipeline(
            outdir=f"{experiment_rootdir}/pipeline",
            max_seqs=100000,
            max_sites=100000,
            armstrong_cutoff=None,
            rate_matrix='input_data/synthetic_rate_matrices/WAG_FastTree.txt',
            n_process=32,
            expected_number_of_MSAs=3912,
            max_families=max_families,
            a3m_dir='input_data/LG_Train_a3m',
            pdb_dir=None,
            use_cached=True,
            num_epochs=200,
            device='cpu',
            center=0.06,
            step_size=0.1,
            n_steps=50,
            keep_outliers=False,
            max_height=1000.0,
            max_path_height=1000,
            precomputed_contact_dir=None,
            precomputed_tree_dir=None,
            precomputed_maximum_parsimony_dir=None,
            edge_or_cherry="cherry",
            init_jtt_ipw=True,
            method=["MLE"],
            xrate_grammar=None,
            fast_tree_cats=4,
            use_site_specific_rates=False,
        )
        pipeline.run()
        learned_rate_matrices['real_data'][max_families] = pipeline.get_learned_Q1()
        num_transitions['real_data'][max_families] = pipeline.get_number_of_single_site_transitions()

        end_to_end_simulator = EndToEndSimulator(
            outdir=f"{experiment_rootdir}/end_to_end_simulator",
            pipeline=pipeline,
            simulation_pct_interacting_positions=0.0,
            use_site_specific_rates_in_simulation=True,  # False, <== Only this chages! (The simulation)
            Q1_ground_truth="input_data/synthetic_rate_matrices/WAG_matrix.txt",
            Q2_ground_truth="input_data/synthetic_rate_matrices/Q2_uniform_constrained.txt",
            fast_tree_rate_matrix='input_data/synthetic_rate_matrices/WAG_FastTree.txt',
            fast_tree_cats=4,
            use_site_specific_rates=False,
            simulate_end_to_end=True,
            simulate_from_trees_wo_ancestral_states=True,
            simulate_from_trees_w_ancestral_states=True,
            use_cached=True,
        )
        end_to_end_simulator.run()
        learned_rate_matrices['w_ancestral_states'][max_families] = end_to_end_simulator.pipeline_on_simulated_data_from_trees_w_ancestral_states.get_learned_Q1()
        learned_rate_matrices['wo_ancestral_states'][max_families] = end_to_end_simulator.pipeline_on_simulated_data_from_trees_wo_ancestral_states.get_learned_Q1()
        learned_rate_matrices['end_to_end'][max_families] = end_to_end_simulator.pipeline_on_simulated_data_end_to_end.get_learned_Q1()
    return learned_rate_matrices, num_transitions
max_families_list = [3912]
learned_rate_matrices_orig_break, num_transitions_orig_break = run_experiment_and_return_learned_rate_matrices()

100%|██████████| 3912/3912 [00:00<00:00, 10615.81it/s]


CPU times: user 3.67 s, sys: 7.16 s, total: 10.8 s
Wall time: 9.91 s


# Now fix the method by taking into account site rate variation

In [ ]:
%%time
from src.pipeline import Pipeline
from src.end_to_end_simulation import EndToEndSimulator
from collections import defaultdict

experiment_rootdir = "cache_LG"

def run_experiment_and_return_learned_rate_matrices():
    learned_rate_matrices = defaultdict(dict)
    num_transitions = defaultdict(dict)
    for max_families in max_families_list:
        pipeline = Pipeline(
            outdir=f"{experiment_rootdir}/pipeline",
            max_seqs=100000,
            max_sites=100000,
            armstrong_cutoff=None,
            rate_matrix='input_data/synthetic_rate_matrices/WAG_FastTree.txt',
            n_process=32,
            expected_number_of_MSAs=3912,
            max_families=max_families,
            a3m_dir='input_data/LG_Train_a3m',
            pdb_dir=None,
            use_cached=True,
            num_epochs=200,
            device='cpu',
            center=0.06,
            step_size=0.1,
            n_steps=50,
            keep_outliers=False,
            max_height=1000.0,
            max_path_height=1000,
            precomputed_contact_dir=None,
            precomputed_tree_dir=None,
            precomputed_maximum_parsimony_dir=None,
            edge_or_cherry="cherry",
            init_jtt_ipw=True,
            method=["MLE"],
            xrate_grammar=None,
            fast_tree_cats=4,
            use_site_specific_rates=True,  # False, <== This should fix it!
        )
        pipeline.run()
        learned_rate_matrices['real_data'][max_families] = pipeline.get_learned_Q1()
        num_transitions['real_data'][max_families] = pipeline.get_number_of_single_site_transitions()

        end_to_end_simulator = EndToEndSimulator(
            outdir=f"{experiment_rootdir}/end_to_end_simulator",
            pipeline=pipeline,
            simulation_pct_interacting_positions=0.0,
            use_site_specific_rates_in_simulation=True,  # False, <== Only this chages! (The simulation)
            Q1_ground_truth="input_data/synthetic_rate_matrices/WAG_matrix.txt",
            Q2_ground_truth="input_data/synthetic_rate_matrices/Q2_uniform_constrained.txt",
            fast_tree_rate_matrix='input_data/synthetic_rate_matrices/WAG_FastTree.txt',
            fast_tree_cats=4,
            use_site_specific_rates=True,  # False, <== This should fix it!
            simulate_end_to_end=True,
            simulate_from_trees_wo_ancestral_states=True,
            simulate_from_trees_w_ancestral_states=True,
            use_cached=True,
        )
        end_to_end_simulator.run()
        learned_rate_matrices['w_ancestral_states'][max_families] = end_to_end_simulator.pipeline_on_simulated_data_from_trees_w_ancestral_states.get_learned_Q1()
        learned_rate_matrices['wo_ancestral_states'][max_families] = end_to_end_simulator.pipeline_on_simulated_data_from_trees_wo_ancestral_states.get_learned_Q1()
        learned_rate_matrices['end_to_end'][max_families] = end_to_end_simulator.pipeline_on_simulated_data_end_to_end.get_learned_Q1()
    return learned_rate_matrices, num_transitions
max_families_list = [3912]
learned_rate_matrices_orig_fixed, num_transitions_orig_fixed = run_experiment_and_return_learned_rate_matrices()

100%|██████████| 3912/3912 [00:00<00:00, 9637.63it/s]


# Analysis code

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import Phylo_util
import numpy as np
import pandas as pd
SRC_DIR = "./"

def normalized(Q):
    pi = Phylo_util.solve_stationery_dist(Q)
    mutation_rate = pi @ -np.diag(Q)
    return Q / mutation_rate

def log_log_plot(Q_true, Q_true_name, Q_pred, Q_pred_name, normalize=False):
    if normalize:
        plt.scatter(np.log(np.abs(normalized(Q_true))), np.log(np.abs(normalized(Q_pred))), alpha=0.3)
        plt.xlabel(f'Normalized {Q_true_name}')
        plt.ylabel(f'Normalized {Q_pred_name}')
    else:
        plt.scatter(np.log(np.abs(Q_true)), np.log(np.abs(Q_pred)), alpha=0.3)
        plt.xlabel(f'{Q_true_name}')
        plt.ylabel(f'{Q_pred_name}')
    plt.xlim([-10, 0])
    plt.ylim([-10, 0])
    plt.plot([-10, 0], [-10, 0], color='r')

def plot_convergence_to_WAG_for_simulated_data(learned_rate_matrices):
    # For the simulated data, plot the ground truth (WAG) against the estimates.
    WAG_matrix = np.array(pd.read_csv(f"{SRC_DIR}input_data/synthetic_rate_matrices/WAG_matrix.txt", sep="\t", index_col=0))
    for max_families in max_families_list:
        print("*" * 15 + f" #families = {max_families} " + "*" * 15)
        for normalize in [True]:
            figure(figsize=(8, 2), dpi=80)
            plt.subplot(1, 3, 1)
            log_log_plot(WAG_matrix, "WAG", learned_rate_matrices['w_ancestral_states'][max_families], "w ancestral states", normalize=normalize)
            plt.subplot(1, 3, 2)
            log_log_plot(WAG_matrix, "WAG", learned_rate_matrices['wo_ancestral_states'][max_families], "wo ancestral states", normalize=normalize)
            plt.subplot(1, 3, 3)
            log_log_plot(WAG_matrix, "WAG", learned_rate_matrices['end_to_end'][max_families], "end-to-end", normalize=normalize)
            plt.tight_layout()
            plt.show()

print("***** Original *****")
plot_convergence_to_WAG_for_simulated_data(learned_rate_matrices_orig)
print("***** Original Break *****")
plot_convergence_to_WAG_for_simulated_data(learned_rate_matrices_orig_break)
print("***** Original Fixed *****")
plot_convergence_to_WAG_for_simulated_data(learned_rate_matrices_orig_fixed)

### Simulation Results w_ancestral_states (==wo_ancestral_states)

In [ ]:
WAG_matrix = np.array(pd.read_csv(f"{SRC_DIR}input_data/synthetic_rate_matrices/WAG_matrix.txt", sep="\t", index_col=0))
normalized(WAG_matrix[:3, :3])

In [ ]:
normalized(learned_rate_matrices_orig['w_ancestral_states'][3912][:3,:3])

In [ ]:
normalized(learned_rate_matrices_orig_break['w_ancestral_states'][3912][:3,:3])  # => This will not converge to WAG because it doesn't use the site rates

In [ ]:
normalized(learned_rate_matrices_orig_fixed['w_ancestral_states'][3912][:3,:3])  # => It is expected that this converges to WAG

### Simulation results end-to-end

In [ ]:
normalized(learned_rate_matrices_orig['end_to_end'][3912][:3,:3])  # => This shows that even wo the trees, our method performs well in simulations without rate variation. Will it perform well _with_ rate variation? See two cells below!

In [ ]:
normalized(learned_rate_matrices_orig_break['end_to_end'][3912][:3,:3])  # => This already did not converge with the trees. We don't expect it to do well here - only worse. As we can see, it performs almost identically.

In [ ]:
normalized(learned_rate_matrices_orig_fixed['end_to_end'][3912][:3,:3])  # => Okay this is the huge one. Because our model now needs to reconstruct the trees AND infer the site rates... Will it do as well as with access to the trees and site rates? It does!

# Let's see what happened on real data

In [ ]:
normalized(learned_rate_matrices_orig['real_data'][3912][:3,:3])

In [ ]:
normalized(learned_rate_matrices_orig_break['real_data'][3912][:3,:3])  # => Should be the same as above, as only the simulation was changed

In [ ]:
normalized(learned_rate_matrices_orig_fixed['real_data'][3912][:3,:3])  # => Looks quite different from our original method which didn't account for site rate variation!